## Shadie single population

In [1]:
import numpy as np
import shadie
import toytree

print("shadie", shadie.__version__)
print("toytree", toytree.__version__)

shadie 0.1.1
toytree 2.1.1-dev


### Create a simple chromosome

In [2]:
chrom = shadie.chromosome.explicit({
    (0, 1_000_000): shadie.NONCDS,
    (1_000_001, 1_200_000): shadie.EXON,
    (1_200_001, 1_300_000): shadie.INTRON,
    (1_300_001, 1_500_000): shadie.EXON,
    (1_500_001, 7_500_000): shadie.NONCDS,
    (7_500_001, 7_700_000): shadie.EXON,
    (7_700_001, 7_800_000): shadie.INTRON,
    (7_800_001, 8_000_000): shadie.EXON,
    (8_000_001, 9_000_000): shadie.NONCDS,
})
chrom.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="150.0px" viewBox="0 0 700.0 150.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t89a3884e772e42dab6c1857c7bcce5a5"> name: noncds
interval: (0, 1000000)
ElementType: g3
coding: False name: exon
interval: (1000001, 1200000)
ElementType: g1
coding: True name: intron
interval: (1200001, 1300000)
ElementType: g2
coding: True name: exon
interval: (1300001, 1500000)
ElementType: g1
coding: True name: noncds
interval: (1500001, 7500000)
ElementType: g3
coding: False name: exon
interval: (7500001, 7700000)
ElementType: g1
coding: True name: intron
interval: (7700001, 7800000)
ElementType: g2
coding: True name: exon
interval: (7800001, 8000000)
ElementType: g1
coding: True name: noncds
interval: (8000001, 9000000)
ElementType: g3
coding: False 0 1000000 2000000 3000000 4000000 5000000 6000000 7000000 8000000 9000000

### Setup models

In [9]:
GENS = 100
POPSIZE = 500
MUT = 1e-8
RECOMB = 1e-8

In [10]:
with shadie.Model() as model_1:
    model_1.initialize(
        chromosome=chrom,
        generations=GENS,
        mut=MUT,
        recomb=RECOMB,
        trees_file="/tmp/test.trees",
    )
    model_1.reproduction.bryophyte("dio", POPSIZE, POPSIZE)

In [11]:
model_1.run(binary="/usr/local/bin/slim", seed=123)

### Recapitate and mutate

In [23]:
import pyslim
import msprime

# load initial SLIM treeseq
slim_ts = pyslim.load("/tmp/test.trees")
slim_ts

In [26]:
# set pop id to 0 for all nodes and reload as a new treeseq
tables = slim_ts.tables
tables.nodes.population = np.zeros(tables.nodes.num_rows, dtype=np.int32)
pts = pyslim.load_tables(tables)

# drop nodes not connected to anyone
nodes_in_edge_table = list(
    set(pts.tables.edges.parent).union(pts.tables.edges.child)
)
pts = pyslim.SlimTreeSequence(
    pts.simplify(samples=nodes_in_edge_table, keep_input_roots=True)
)

In [27]:
# show treeseq
pts

In [40]:
# recapitate until all coalesced.
rts = pyslim.recapitate(pts, ancestral_Ne=POPSIZE, recombination_rate=RECOMB, random_seed=6)

# add neutral mutations
rts = msprime.sim_mutations(rts, rate=MUT, model=msprime.SLiMMutationModel(type=0))

In [41]:
rts

### Load as ToyTreeSequence for drawing

In [42]:
tts = toytree.utils.tree_sequence.ToyTreeSequence(rts, sample=10, seed=123)

In [45]:
# draw a single tree
tts.draw_tree(idx=0, layout='d', scale_bar=True);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="270.0px" viewBox="0 0 350.0 270.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t19e9aa85e1c648d7a28a9bf9f4db7d14"> p0-1 p0-0 p0-3 p0-5 p0-9 p0-7 p0-6 p0-4 p0-8 p0-2 id: 0
pos: 1510
time: 44
mtype: 0 id: 1
pos: 2342
time: 1108
mtype: 0 0 500 1000 tree in interval 0 (sites 0-8436)

In [46]:
# draw sequence
tts.draw_tree_sequence(width=1200);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="1200.0px" height="325.0px" viewBox="0 0 1200.0 325.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb7d480d519694292ab211bef17a4350c"> 0 1000 2000 <polygon points="50.0,56.86394142769982 59.82142857142857,84.3197071384991 59.82142857142857,221.59853569249543 167.85714285714286,221.59853569249543 167.85714285714286,84.3197071384991 77.4889091689374,56.86394142769982 77.4889091689374,56.86394142769982 167.85714285714286,56.86394142769982 167.85714285714286,56.86394142769982 59.82142857142857,56.86394142769982 59.82142857142857,56.86394142769982 50.0,56.86394142769982" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 0
interval: (0 - 8436)
tmrca: 1127.04 <polygon points="77.4889091689374,56.86394142769982 167.85714285714286,84.3197071384991 167.85714285714286,221.59853569249543 275.8928571428571,221.59853569249543 275.8928571428571,84.3197071384991 293.3042177473979,56.86394142769982 293.3042177473979,56.86394142769982 275.8928571428571,56.86394142769982 275.8928571428571,56.86394142769982 167.85714285714286,56.86394142769982 167.85714285714286,56.86394142769982 77.4889091689374,56.86394142769982" style="fill:rgb(98.8%,55.3%,38.4%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 1
interval: (8436 - 74667)
tmrca: 1127.04 <polygon points="293.3042177473979,56.86394142769982 275.8928571428571,84.3197071384991 275.8928571428571,221.59853569249543 383.9285714285714,221.59853569249543 383.9285714285714,84.3197071384991 323.23376145492404,56.86394142769982 323.23376145492404,56.86394142769982 383.9285714285714,56.86394142769982 383.9285714285714,56.86394142769982 275.8928571428571,56.86394142769982 275.8928571428571,56.86394142769982 293.3042177473979,56.86394142769982" style="fill:rgb(55.3%,62.7%,79.6%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 2
interval: (74667 - 83852)
tmrca: 1312.77 <polygon points="323.23376145492404,56.86394142769982 383.9285714285714,84.3197071384991 383.9285714285714,221.59853569249543 491.9642857142857,221.59853569249543 491.9642857142857,84.3197071384991 535.3082843854542,56.86394142769982 535.3082843854542,56.86394142769982 491.9642857142857,56.86394142769982 491.9642857142857,56.86394142769982 383.9285714285714,56.86394142769982 383.9285714285714,56.86394142769982 323.23376145492404,56.86394142769982" style="fill:rgb(90.6%,54.1%,76.5%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 3
interval: (83852 - 148935)
tmrca: 1210.19 idx: 4
interval: (148935 - 175639)
tmrca: 1312.77 idx: 5
interval: (175639 - 181381)
tmrca: 1312.77 <polygon points="641.034356800739,56.86394142769982 708.0357142857142,84.3197071384991 708.0357142857142,221.59853569249543 816.0714285714286,221.59853569249543 816.0714285714286,84.3197071384991 944.9471225024087,56.86394142769982 944.9471225024087,56.86394142769982 816.0714285714286,56.86394142769982 816.0714285714286,56.86394142769982 708.0357142857142,56.86394142769982 708.0357142857142,56.86394142769982 641.034356800739,56.86394142769982" style="fill:rgb(89.8%,76.9%,58%);fill-opacity:0.25;opacity:1.0;stroke:rgb(100%,100%,100%);stroke-opacity:0.5"> idx: 6
interval: (181381 - 274648)
tmrca: 1551.47 idx: 7
interval: (274648 - 291973)
tmrca: 1971.56 <polygon points="1001.4010522501374,56.86394142769982 924.1071428571428,84.3197071384991 924.1071428571428,221.59853569249543 1032.1428571428573,221.59853569249543 1032.1428571428573,84.3197071384991 1051.5138941312528,56.86394142769982 1051.5138941312528,56.86